In [3]:
import os
import glob
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import re

# ============================================================
# STEP 1: CLEAN FILES
# ============================================================
def clean_task1_dataset(raw_folder, clean_folder):
    os.makedirs(clean_folder, exist_ok=True)
    all_files = glob.glob(f"{raw_folder}/*.txt")
    valid = 0

    for f in all_files:
        try:
            with open(f, 'r', errors='ignore') as file:
                lines = file.readlines()

            if not lines:
                continue

            lines = lines[1:]  # skip point count
            lines = [ln for ln in lines if ln.strip()]
            if len(lines) < 6:
                continue

            clean_path = f.replace(raw_folder, clean_folder)
            os.makedirs(os.path.dirname(clean_path), exist_ok=True)

            with open(clean_path, 'w') as wf:
                wf.writelines(lines)

            valid += 1
        except:
            continue

    print(f"✅ Cleaned and saved {valid} valid signature files.")



# ============================================================
# STEP 2: FEATURE EXTRACTION
# ============================================================
def extract_features_from_signature(df):
    try:
        df = df.iloc[:, :3].copy()
        df.columns = ["X", "Y", "T"]
        df = df.apply(pd.to_numeric, errors="coerce").dropna()

        if len(df) < 6:
            return None

        dx = np.diff(df["X"])
        dy = np.diff(df["Y"])
        dt = np.diff(df["T"])
        dt[dt == 0] = 1

        # Velocity
        v = np.sqrt(dx**2 + dy**2) / dt

        # Acceleration
        a = np.diff(v) / dt[1:]

        # Direction
        theta = np.arctan2(dy, dx)

        # Angular velocity
        w = np.diff(theta) / dt[1:]

        # Angular acceleration
        alpha = np.diff(w) / dt[2:]

        # Simulated pressure
        pressure = 1 / (v + 1e-6)

        # Torque proxy
        tau = pressure * v

        def stats(arr):
            return np.mean(arr), np.std(arr), np.max(arr)

        stroke_count = np.sum(dt > np.percentile(dt, 90)) + 1
        direction_changes = np.sum(np.abs(np.diff(theta)) > np.pi / 4)

        return {
            "v_mean": stats(v)[0],
            "v_std": stats(v)[1],
            "v_max": stats(v)[2],

            "a_mean": stats(a)[0],
            "a_std": stats(a)[1],
            "a_max": stats(a)[2],

            "w_mean": stats(w)[0],
            "w_std": stats(w)[1],

            "alpha_mean": stats(alpha)[0],
            "alpha_std": stats(alpha)[1],

            "tau_mean": stats(tau)[0],
            "tau_std": stats(tau)[1],
            "tau_max": stats(tau)[2],

            "stroke_count": stroke_count,
            "direction_changes": direction_changes
        }

    except Exception as e:
        return None



# ============================================================
# STEP 3: BUILD DATASET WITH CORRECT LABELS
# ============================================================
def build_training_dataset(clean_folder):
    files = glob.glob(f"{clean_folder}/*.txt")
    rows = []
    genuine_count = forgery_count = 0

    for f in files:
        try:
            fname = os.path.basename(f)
            match = re.match(r'U(\d+)S(\d+)\.txt', fname, re.IGNORECASE)
            if not match:
                continue

            instance_id = int(match.group(2))

            df = pd.read_csv(f, sep=r"\s+", header=None)
            features = extract_features_from_signature(df)

            if features:
                if 1 <= instance_id <= 20:
                    features["label"] = 1
                    genuine_count += 1
                else:
                    features["label"] = 0
                    forgery_count += 1

                rows.append(features)

        except:
            continue

    feature_df = pd.DataFrame(rows)
    feature_df.to_csv("physics_signature_features.csv", index=False)

    print(f"✅ Extracted {len(feature_df)} samples")
    print(f"   Genuine: {genuine_count} | Forged: {forgery_count}")

    return feature_df


# ============================================================
# STEP 4: TRAIN RANDOM FOREST
# ============================================================
def train_signature_model(df):
    if df.empty or len(df["label"].unique()) < 2:
        print("⚠ Not enough data to train.")
        return

    X = df.drop("label", axis=1)
    y = df["label"]

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

    model = RandomForestClassifier(
        n_estimators=300,
        max_depth=12,
        random_state=42
    )

    model.fit(X_train, y_train)

    preds = model.predict(X_test)

    print("\n✅ PHYSICS-BASED RF TRAINED")
    print("Accuracy:", accuracy_score(y_test, preds))
    print("\nReport:\n", classification_report(y_test, preds))
    print("\nConfusion Matrix:\n", confusion_matrix(y_test, preds))


# ============================================================
# MAIN EXECUTION
# ============================================================
RAW_PATH = r"C:\Users\Nandini\osv-hybrid\notebooks\Task1"
CLEAN_PATH = r"C:\Users\Nandini\osv-hybrid\notebooks\Task1_clean"

clean_task1_dataset(RAW_PATH, CLEAN_PATH)
df = build_training_dataset(CLEAN_PATH)
train_signature_model(df)


✅ Cleaned and saved 1600 valid signature files.
✅ Extracted 1600 samples
   Genuine: 800 | Forged: 800

✅ PHYSICS-BASED RF TRAINED
Accuracy: 0.76875

Report:
               precision    recall  f1-score   support

           0       0.77      0.78      0.78       166
           1       0.76      0.75      0.76       154

    accuracy                           0.77       320
   macro avg       0.77      0.77      0.77       320
weighted avg       0.77      0.77      0.77       320


Confusion Matrix:
 [[130  36]
 [ 38 116]]


In [ ]:
# ============================================================
# STATIC SIGNATURE VERIFICATION - SIAMESE CNN (SINGLE FILE)
# ============================================================

import os
import cv2
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, backend as K
from tensorflow.keras.applications import MobileNetV2

# ============================================================
# CONFIG
# ============================================================

DATASET_PATH = r""   # CHANGE THIS
IMG_SIZE = 128
BATCH_SIZE = 16
EPOCHS = 10

# ============================================================
# IMAGE LOADER
# ============================================================

def load_image(path):
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
    img = img / 255.0
    return img.reshape(IMG_SIZE, IMG_SIZE, 1)

# ============================================================
# PAIR GENERATOR
# ============================================================

def generate_pairs(root):
    users = os.listdir(root)
    pairs, labels = [], []

    for user in users:
        user_path = os.path.join(root, user)
        imgs = os.listdir(user_path)

        for _ in range(5):
            # Genuine pair
            i1, i2 = random.sample(imgs, 2)
            pairs.append((os.path.join(user_path, i1),
                          os.path.join(user_path, i2)))
            labels.append(1)

            # Forged pair
            other = random.choice([u for u in users if u != user])
            other_img = random.choice(os.listdir(os.path.join(root, other)))
            pairs.append((os.path.join(user_path, i1),
                          os.path.join(root, other, other_img)))
            labels.append(0)

    return pairs, labels

pairs, labels = generate_pairs(DATASET_PATH)

# ============================================================
# DATASET PREPARATION
# ============================================================

X1, X2, y = [], [], []

for (p1, p2), label in zip(pairs, labels):
    X1.append(load_image(p1))
    X2.append(load_image(p2))
    y.append(label)

X1 = np.array(X1)
X2 = np.array(X2)
y = np.array(y)

# ============================================================
# EMBEDDING NETWORK (MobileNet)
# ============================================================

def embedding_network():
    base = MobileNetV2(
        input_shape=(IMG_SIZE, IMG_SIZE, 1),
        include_top=False,
        weights=None
    )

    x = base.output
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(128, activation="relu")(x)
    x = layers.Lambda(lambda x: K.l2_normalize(x, axis=1))(x)

    return models.Model(base.input, x)

# ============================================================
# SIAMESE MODEL
# ============================================================

def euclidean_distance(v):
    a, b = v
    return K.sqrt(K.sum(K.square(a - b), axis=1, keepdims=True))

input_a = layers.Input((IMG_SIZE, IMG_SIZE, 1))
input_b = layers.Input((IMG_SIZE, IMG_SIZE, 1))

embed = embedding_network()

ea = embed(input_a)
eb = embed(input_b)

dist = layers.Lambda(euclidean_distance)([ea, eb])
output = layers.Dense(1, activation="sigmoid")(dist)

model = models.Model([input_a, input_b], output)

model.compile(
    loss="binary_crossentropy",
    optimizer=tf.keras.optimizers.Adam(1e-4),
    metrics=["accuracy"]
)

# ============================================================
# TRAIN
# ============================================================

model.fit([X1, X2], y, batch_size=BATCH_SIZE, epochs=EPOCHS)

model.save("siamese_static_model.h5")

print("✅ STATIC SIAMESE CNN TRAINED SUCCESSFULLY")

# ============================================================
# EMBEDDING COMPARISON FUNCTION (INFERENCE)
# ============================================================

def compare_signatures(img1, img2):
    embed_model = embedding_network()
    embed_model.set_weights(model.get_weights())

    e1 = embed_model(load_image(img1).reshape(1, IMG_SIZE, IMG_SIZE, 1))
    e2 = embed_model(load_image(img2).reshape(1, IMG_SIZE, IMG_SIZE, 1))

    return np.linalg.norm(e1 - e2)

# Example:
# score = compare_signatures("sig1.png", "sig2.png")
# print("Similarity Distance:", score)


SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (1147917952.py, line 17)

In [ ]:
# ============================================================
# SIAMESE CNN FOR GPDS (FINAL - FULL UPDATED CODE)
# ============================================================

import os
import cv2
import re
import random
#import pickle
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, backend as K
from tensorflow.keras.applications import MobileNetV2

# ============================================================
# CONFIGURATION
# ============================================================

GPDS_TRAIN_PATH = r"C:\Users\Nandini\osv-hybrid\training\New folder (10)\train"   
IMG_SIZE = 96
EPOCHS = 10
BATCH_SIZE = 32


# ============================================================
# IMAGE PREPROCESSING FUNCTION
# ============================================================

def load_image(path):
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
    img = img / 255.0
    return img.reshape(IMG_SIZE, IMG_SIZE, 1)

# ============================================================
# LOAD FILES
# ============================================================

genuine_path = os.path.join(GPDS_TRAIN_PATH, "genuine")
forge_path   = os.path.join(GPDS_TRAIN_PATH, "forge")

genuine_files = os.listdir(genuine_path)
forge_files   = os.listdir(forge_path)

# ============================================================
# USER ID EXTRACTION (FOR c-001-01, cf-001-01 FORMAT)
# ============================================================

def get_uid(filename):
    match = re.search(r'-(\d{3})-', filename)
    return match.group(1) if match else None

# ============================================================
# GENERATE SIAMESE PAIRS
# ============================================================

X1, X2, y = [], [], []

for g in genuine_files:
    uid = get_uid(g)
    if uid is None:
        continue

    same_user_genuine = [
        f for f in genuine_files
        if get_uid(f) == uid and f != g
    ]

    same_user_forged = [
        f for f in forge_files
        if get_uid(f) == uid
    ]

    if not same_user_genuine or not same_user_forged:
        continue

    # -------- GENUINE PAIR --------
    g2 = random.choice(same_user_genuine)
    X1.append(load_image(os.path.join(genuine_path, g)))
    X2.append(load_image(os.path.join(genuine_path, g2)))
    y.append(1)

    # -------- FORGED PAIR --------
    f2 = random.choice(same_user_forged)
    X1.append(load_image(os.path.join(genuine_path, g)))
    X2.append(load_image(os.path.join(forge_path, f2)))
    y.append(0)

X1 = np.array(X1)
X2 = np.array(X2)
y  = np.array(y).astype("float32")

print("Shapes:", X1.shape, X2.shape, y.shape)

# ============================================================
# EMBEDDING MODEL (MOBILENET BASED)
# ============================================================

def embedding_model():
    base = MobileNetV2(
        input_shape=(IMG_SIZE, IMG_SIZE, 1),
        include_top=False,
        weights=None
    )
    for layer in base.layers:
        layer.trainable = False

    x = base.output
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(128, activation="relu")(x)
    class L2Normalize(layers.Layer):
        def call(self, inputs):
            return tf.math.l2_normalize(inputs, axis=1)
    x = L2Normalize()(x)


    return models.Model(base.input, x)

# ============================================================
# SIAMESE NETWORK (DISTANCE OUTPUT)
# ============================================================

# ============================================================
# CUSTOM DISTANCE LAYER (KERAS 3 SAFE)
# ============================================================

class EuclideanDistance(layers.Layer):
    def call(self, inputs):
        a, b = inputs
        return tf.sqrt(tf.reduce_sum(tf.square(a - b), axis=1, keepdims=True))


input_a = layers.Input((IMG_SIZE, IMG_SIZE, 1))
input_b = layers.Input((IMG_SIZE, IMG_SIZE, 1))

embed = embedding_model()

ea = embed(input_a)
eb = embed(input_b)

distance = EuclideanDistance()([ea, eb])

siamese = models.Model([input_a, input_b], distance)


# ============================================================
# CONTRASTIVE LOSS (CORRECT FOR SIAMESE)
# ============================================================

def contrastive_loss(y_true, d):
    margin = 1.0
    return K.mean(
        y_true * K.square(d) +
        (1 - y_true) * K.square(K.maximum(margin - d, 0))
    )

siamese.compile(
    loss=contrastive_loss,
    optimizer=tf.keras.optimizers.Adam(1e-4)
)

# ============================================================
# TRAIN MODEL
# ============================================================

siamese.fit(
    [X1, X2],
    y,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE
)

print("✅ Siamese CNN trained successfully")

# ============================================================
# SAVE EMBEDDING MODEL AS .PKL (BACKEND USE)
# ============================================================

# build embedding network
embedding_net = embedding_model()

# copy weights
embedding_net.set_weights(embed.get_weights())

# ✅ SAVE AS .h5
embedding_net.save("static_embedding_model.h5")

print("✅ Embedding model saved as static_embedding_model.h5")


Shapes: (4800, 96, 96, 1) (4800, 96, 96, 1) (4800,)
Epoch 1/3
150/150 ━━━━━━━━━━━━━━━━━━━━ 54s 322ms/step - loss: 0.5000
Epoch 2/3
150/150 ━━━━━━━━━━━━━━━━━━━━ 34s 223ms/step - loss: 0.5000
Epoch 3/3
150/150 ━━━━━━━━━━━━━━━━━━━━ 30s 201ms/step - loss: 0.5000
✅ Siamese CNN trained successfully


✅ Embedding model saved as static_embedding_model.h5


In [1]:
# ============================================================
# SIAMESE CNN FOR GPDS (KERAS 3 SAFE - FINAL)
# ============================================================

import os
import cv2
import re
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, backend as K
from tensorflow.keras.applications import MobileNetV2

# ============================================================
# CONFIG
# ============================================================

GPDS_TRAIN_PATH = r"C:\Users\Nandini\osv-hybrid\training\New folder (10)\train"
IMG_SIZE = 96
EPOCHS = 10
BATCH_SIZE = 32

# ============================================================
# IMAGE LOADER
# ============================================================

def load_image(path):
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
    img = img / 255.0
    return img.reshape(IMG_SIZE, IMG_SIZE, 1)

# ============================================================
# LOAD FILES
# ============================================================

genuine_path = os.path.join(GPDS_TRAIN_PATH, "genuine")
forge_path   = os.path.join(GPDS_TRAIN_PATH, "forge")

genuine_files = os.listdir(genuine_path)
forge_files   = os.listdir(forge_path)

def get_uid(filename):
    m = re.search(r'-(\d{3})-', filename)
    return m.group(1) if m else None

# ============================================================
# BUILD PAIRS
# ============================================================

X1, X2, y = [], [], []

for g in genuine_files:
    uid = get_uid(g)
    if uid is None:
        continue

    pos = [f for f in genuine_files if get_uid(f) == uid and f != g]
    neg = [f for f in forge_files if get_uid(f) == uid]

    if not pos or not neg:
        continue

    g2 = random.choice(pos)
    f2 = random.choice(neg)

    X1.append(load_image(os.path.join(genuine_path, g)))
    X2.append(load_image(os.path.join(genuine_path, g2)))
    y.append(1)

    X1.append(load_image(os.path.join(genuine_path, g)))
    X2.append(load_image(os.path.join(forge_path, f2)))
    y.append(0)

X1 = np.array(X1)
X2 = np.array(X2)
y  = np.array(y).astype("float32")

print("Pairs:", X1.shape, X2.shape)

# ============================================================
# EMBEDDING MODEL (NO CUSTOM LAYERS)
# ============================================================

def embedding_model():
    base = MobileNetV2(
        input_shape=(IMG_SIZE, IMG_SIZE, 1),
        include_top=False,
        weights=None
    )
    for layer in base.layers:
        layer.trainable = False

    x = base.output
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(128, activation="relu")(x)

    return models.Model(base.input, x)

# ============================================================
# SIAMESE NETWORK (TRAINING ONLY)
# ============================================================

def euclidean_distance(v):
    a, b = v
    return K.sqrt(K.sum(K.square(a - b), axis=1, keepdims=True))

input_a = layers.Input((IMG_SIZE, IMG_SIZE, 1))
input_b = layers.Input((IMG_SIZE, IMG_SIZE, 1))

embed = embedding_model()

ea = embed(input_a)
eb = embed(input_b)

distance = layers.Lambda(euclidean_distance)([ea, eb])

siamese = models.Model([input_a, input_b], distance)

def contrastive_loss(y_true, d):
    margin = 1.0
    return K.mean(
        y_true * K.square(d) +
        (1 - y_true) * K.square(K.maximum(margin - d, 0))
    )

siamese.compile(
    loss=contrastive_loss,
    optimizer=tf.keras.optimizers.Adam(1e-4)
)

# ============================================================
# TRAIN
# ============================================================

siamese.fit(
    [X1, X2],
    y,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE
)

print("✅ Siamese trained")

# ============================================================
# SAVE EMBEDDING MODEL (PRODUCTION)
# ============================================================

embedding_net = embedding_model()
embedding_net.set_weights(embed.get_weights())

# 🔥 SAVE AS .keras (NO ERRORS EVER)
embedding_net.save("static_embedding_model.keras")

print("✅ Embedding model saved (static_embedding_model.keras)")

Pairs: (4800, 96, 96, 1) (4800, 96, 96, 1)

Epoch 1/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 19s 99ms/step - loss: 0.5000
Epoch 2/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 19s 130ms/step - loss: 0.5000
Epoch 3/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 20s 130ms/step - loss: 0.5000
Epoch 4/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 21s 138ms/step - loss: 0.5000
Epoch 5/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 40s 128ms/step - loss: 0.5000
Epoch 6/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 21s 130ms/step - loss: 0.5000
Epoch 7/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 20s 133ms/step - loss: 0.5000
Epoch 8/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 16s 108ms/step - loss: 0.5000
Epoch 9/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 13s 90ms/step - loss: 0.5000
Epoch 10/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 14s 96ms/step - loss: 0.5000
✅ Siamese trained
✅ Embedding model saved (static_embedding_model.keras)
